# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

In [2]:
#show me the CSV!
df = pd.read_csv('output_data/cities.csv')
df

,Unnamed: 0,City,Lat,Lng,Temp,Humidity,Cloudiness,Wind,Country,Date
0,0,vila velha,-20.3297,-40.2925,86.00,66,20,12.66,BR,1612639871
1,1,albany,42.6001,-73.9662,37.40,35,75,13.80,US,1612639930
2,2,tiruvidaimarudur,10.9960,79.4521,68.00,89,0,7.40,IN,1612639930
3,3,ushuaia,-54.8000,-68.3000,55.40,67,20,21.85,AR,1612639930
4,4,bethel,41.3712,-73.4140,42.80,31,40,13.80,US,1612639660
...,...,...,...,...,...,...,...,...,...,...
558,558,san vicente,-26.6167,-54.1333,79.93,37,0,6.53,AR,1612639994
559,559,tessalit,20.1986,1.0114,75.96,17,71,9.46,ML,1612639994
560,560,chara,56.9064,118.2631,-14.46,73,100,0.54,RU,1612639705
561,561,buin,-33.7333,-70.7500,71.60,60,0,8.05,CL,1612639994


In [3]:
#grab locations by lat and long
locations = df[["Lat", "Lng"]]

#grab humidity column for weight
weight = df['Humidity']

In [5]:
#plot heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=weight,
                                 dissipating=False, max_intensity=300,
                                 point_radius=5)
# Add layer
fig.add_layer(heat_layer)

plt.savefig('output_data/heat_map.png')

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>

In [6]:
#narrow down the cities by desired weather conditions
narrow_cities =  df.loc[(df["Humidity"] < 80) & (df["Humidity"] > 10) & (df["Cloudiness"] == 0) & (df["Temp"] > 75.00) & (df["Temp"] < 80.00) & (df["Wind"] < 10.00)].reset_index(drop=True)
del narrow_cities["Unnamed: 0"]
narrow_cities

,City,Lat,Lng,Temp,Humidity,Cloudiness,Wind,Country,Date
0,castro,-24.7911,-50.0119,75.83,54,0,4.32,BR,1612639934
1,la reforma,25.0833,-108.0500,77.00,48,0,4.00,MX,1612639957
2,nkoteng,4.5167,12.0333,76.86,27,0,2.86,CM,1612639964
3,oussouye,12.4850,-16.5469,77.00,57,0,9.22,SN,1612639970
4,viedma,-40.8135,-62.9967,78.01,45,0,1.01,AR,1612639975
5,moissala,8.3404,17.7663,78.85,17,0,4.27,TD,1612639977
6,san vicente,-26.6167,-54.1333,79.93,37,0,6.53,AR,1612639994


In [7]:
#add a new column to hold hotel names
hotel_df = narrow_cities
hotel_df["Hotel Name"] = ""
hotel_df

,City,Lat,Lng,Temp,Humidity,Cloudiness,Wind,Country,Date,Hotel Name
0,castro,-24.7911,-50.0119,75.83,54,0,4.32,BR,1612639934,
1,la reforma,25.0833,-108.0500,77.00,48,0,4.00,MX,1612639957,
2,nkoteng,4.5167,12.0333,76.86,27,0,2.86,CM,1612639964,
3,oussouye,12.4850,-16.5469,77.00,57,0,9.22,SN,1612639970,
4,viedma,-40.8135,-62.9967,78.01,45,0,1.01,AR,1612639975,
5,moissala,8.3404,17.7663,78.85,17,0,4.27,TD,1612639977,
6,san vicente,-26.6167,-54.1333,79.93,37,0,6.53,AR,1612639994,


In [8]:
# params dictionary to update each iteration
params = {
    "radius": 50000,
    "types": "lodging",
    "key": g_key
}

# Use the lat/lng we recovered to identify hotels in the area
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)

    # convert to json
    name_address = name_address.json()

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        #store the name via path into hotel name column
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]

    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
hotel_df

Missing field/result... skipping.


,City,Lat,Lng,Temp,Humidity,Cloudiness,Wind,Country,Date,Hotel Name
0,castro,-24.7911,-50.0119,75.83,54,0,4.32,BR,1612639934,Hotel Ponta Grossa Plateau
1,la reforma,25.0833,-108.0500,77.00,48,0,4.00,MX,1612639957,Motel Polvorín
2,nkoteng,4.5167,12.0333,76.86,27,0,2.86,CM,1612639964,Sanaga Beach Lodges
3,oussouye,12.4850,-16.5469,77.00,57,0,9.22,SN,1612639970,La Tortue Bleue
4,viedma,-40.8135,-62.9967,78.01,45,0,1.01,AR,1612639975,Hotel Cristal
5,moissala,8.3404,17.7663,78.85,17,0,4.27,TD,1612639977,
6,san vicente,-26.6167,-54.1333,79.93,37,0,6.53,AR,1612639994,Posada La Bonita


In [9]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Create a marker_layer using the hotel name list to fill the info box
fig = gmaps.figure()
markers = gmaps.marker_layer(locations,
    info_box_content=[f" {name}" for name in hotel_info])

#combine figures
fig.add_layer(markers)
fig.add_layer(heat_layer)

# Display figures
fig

Figure(layout=FigureLayout(height='420px'))